In [69]:
import Levenshtein

In [70]:
from seqtools import *                                                                                                                                                           
import configargparse                                                                                                                                                            
import pandas as pd                                                                                                                                                              
import torch                                                                                                                                                                     
import torch.autograd as autograd                                                                                                                                                
import torch.nn as nn                                                                                                                                                            
import torch.nn.functional as F                                                                                                                                                  
import torch.optim as optim                                                                                                                                                      
import pandas as pd                                                                                                                                                              
torch.manual_seed(1)                                                                                                                                                             
from torch.autograd import Variable                                                                                                                                              
import numpy as np                                                                                                                                                               
import sys                                                                                                                                                                       
# Minimum count                                                                                                                                                                  
MIN_COUNT = 1                                                                                                                                                                    
# Min length of input and output sequence                                                                                                                                        
MIN_LENGTH = 1                                                                                                                                                                   
# Max length of input and output sequence                                                                                                                                        
MAX_LENGTH = 30   
import Levenshtein

In [71]:
import kenlm
lmodel = kenlm.Model('./data/wordlist_english_filtered_threshold100-kenlm.arpa')

In [72]:
from lstm_model import LSTMTagger, filter_pairs, prepare_data, read_langs, get_validation_batch

In [40]:
df = pd.read_csv('./seq2seq_output/global_candidates.phonemes', sep='\t', skiprows=3, names=['word','phonemes'])
df=df.dropna()

import kenlm

phoneme_model = kenlm.Model('./data/cmudict-ud.arpa')

def phoneme_score(s):
    return phoneme_model.score(s)

def phoneme_len(s):
    return len(s.split(' '))

df["score"]=df["phonemes"].apply(phoneme_score)
df["len"]=df["phonemes"].apply(phoneme_len)
df["nscore"]=df["score"]/(df["len"]**0.4)

df.sort_values(by='nscore', ascending=True)

phoneme_score_dict=dict(zip(df.word.values, df.nscore.values))

In [41]:
def encode_word(examples):
    seqs=[]
    for example in examples:
        seqs.append(indexes_from_sentence(input_lang, example))
    seqs_padded = [pad_seq(seq, MAX_LENGTH) for seq in seqs]
        # Turn padded arrays into (batch_size x max_len) tensors, transpose into (max_len x batch_size)                                                                              
    input_var = Variable(torch.LongTensor(seqs_padded)).transpose(0, 1)                                                                                                         
    if USE_CUDA:                                                                                                                                                                 
        input_var = input_var.cuda()                                                                                                                                                                                                                                                                                                                            
    return input_var                                                                                                                                                                                           

In [42]:
def load_model(embedding_dim, hidden_dim, vocab_size, tagset_size, batch_size, model_filename, n_layers=1):
    t=torch.load(model_filename)
    m=LSTMTagger(embedding_dim=embedding_dim,hidden_dim=hidden_dim,
                 vocab_size=vocab_size,tagset_size=tagset_size,batch_size=batch_size, n_layers=n_layers)
    m.load_state_dict(t)
    m.hidden=m.init_hidden(batch_size=m.batch_size)
    if USE_CUDA:
        m.cuda()
    return m

In [43]:
def get_from_copy_edit(example, copyedits):
    output_str=[]
    for i,s in enumerate(list(example)):
        if copyedits[i] == 'C':
            output_str.append(s)
    return u''.join(output_str)

In [44]:
def get_predict_raw(m, example):  
    import itertools
    outputs=[]
    m.hidden=m.init_hidden(batch_size=m.batch_size)
    t = encode_word(list(itertools.repeat(example, m.batch_size)))
    scores = m(t)
    scores=F.log_softmax(scores[0,:]) #do a softmax since LSTM onlt returns linear output out.
    return scores.cpu().data.numpy()
    #return scores[0,:,:].cpu().data.numpy()

In [45]:
def get_predict(m, examples):  
    import itertools
    outputs=[]
    for example in examples:
        m.hidden=m.init_hidden(batch_size=m.batch_size)
        t = encode_word(list(itertools.repeat(example, m.batch_size)))
        scores = m(t)
        preds=[output_lang.index2word[ix] for ix in np.argmax(scores[0,:,:].cpu().data.numpy(), axis=1)]
        output_str=get_from_copy_edit(example, preds)
        outputs.append(u''.join(output_str))
    return outputs

In [46]:
def score_candidate_length(c, example,length_model):
    import scipy.stats
    needed_length, needed_length_std=length_model.predict(np.array([[len(example)]]), return_std=True)
    needed_length=needed_length[0]
    needed_length_std=needed_length_std[0]
    clength=len(c)
    #print "NL:{} CL:{}".format(needed_length, clength)
    return scipy.stats.norm.logpdf(clength, loc=needed_length, scale=needed_length_std)-scipy.stats.norm.logpdf(needed_length, loc=needed_length, scale=needed_length_std)    

In [47]:
def get_predict_lm(m, examples, lmodel, length_model, alpha=0.5, use_lm=False, use_pm=False, use_length=False):  
    import itertools
    outputs=[]
    preds=[]
    odflist=[]
    for example in examples:
        m.hidden=m.init_hidden(batch_size=m.batch_size)
        t = encode_word(list(itertools.repeat(example, m.batch_size)))
        scores = m(t)
        scores=F.log_softmax(scores[0,:]).cpu().data.numpy()
        candidates, lstm_scores=zip(*get_k_best_blends(scores, example, k=5))
        lm_scores=[lmodel.score(' '.join(list(c)))/(float(len(c))**alpha) for c in candidates]
        pm_scores=[]
        for c in candidates:
            if c in phoneme_score_dict:
                pm_scores.append(phoneme_score_dict[c])
            else:
                pm_scores.append(0.0)
        length_scores=[score_candidate_length(c, example, length_model) for c in candidates]
        odf=pd.DataFrame().from_records(zip(candidates, lstm_scores,lm_scores, pm_scores, length_scores), columns=['candidate','lstm','lm', 'pm','lengthscore'])
        odf["total"]=odf["lstm"]
        if use_lm:
            odf["total"]=odf["total"]+odf["lm"]
        if use_pm:
            odf["total"]=odf["total"]+odf["pm"]
        if use_length:
            odf["total"]=odf["total"]+odf["lengthscore"]
        odf=odf.sort_values(by=['total'], ascending=False)
        odf=odf.reset_index()
        odflist.append(odf)
        preds.append(odf.ix[0].candidate)
    return preds, pd.concat(odflist)

In [48]:
from collections import Counter
class Pmf(Counter):
    """A Counter with probabilities."""

    def normalize(self):
        """Normalizes the PMF so the probabilities add to 1."""
        total = float(sum(self.values()))
        for key in self:
            self[key] /= total

    def __add__(self, other):
        """Adds two distributions.

        The result is the distribution of sums of values from the
        two distributions.

        other: Pmf

        returns: new Pmf
        """
        pmf = Pmf()
        for key1, prob1 in self.items():
            for key2, prob2 in other.items():
                pmf[key1 + key2] += prob1 * prob2
        return pmf

    def __hash__(self):
        """Returns an integer hash value."""
        return id(self)
    
    def __eq__(self, other):
        return self is other

    def render(self):
        """Returns values and their probabilities, suitable for plotting."""
        return zip(*sorted(self.items()))

In [49]:
def only_length(split_file_name, length_model_name):
    from collections import Counter
    _, _,train_pairs,val_pairs,t_pairs=pickle.load(open(split_file_name,'rb'))
    length_model = pickle.load(open(length_model_name,'rb'))
    prefixes=[t.find('D') for t in zip(*train_pairs)[1]]
    cprefixes=Counter(prefixes)
    cpmf=Pmf(cprefixes)
    cpmf.normalize()
    cpmf_dist_keys, cpmf_dist_values=zip(*sorted(cpmf.items()))    
    #print cpmf_dist_keys, cpmf_dist_values
    preds=[]
    test_pairs=zip(*t_pairs)[0]
    for t in test_pairs:
        tlen, tstd=length_model.predict(np.array([len(t)]).reshape(-1,1), return_std=True)
        tlen_sample = int(np.random.normal(loc=tlen, scale=tstd))
        if tlen_sample > len(t):
            tlen_sample = len(t)
        len_prefix=np.random.choice(len(cpmf_dist_values), p=cpmf_dist_values)
        #print "Length", len_prefix
        pb, sb = t.split('}')
        p1blend=pb[:len_prefix]
        if len(p1blend) < tlen_sample:
            s1blend=sb[-(tlen_sample-len_prefix):]
        else:
            s1blend=''
        preds.append(p1blend+s1blend)
    gold_preds=[get_from_copy_edit(val_pair[0], val_pair[1]) for val_pair in t_pairs]
    edit_scores=[Levenshtein.distance(p,g) for p, g in zip(preds, gold_preds)]

    return np.mean(edit_scores), preds, gold_preds, edit_scores, None

In [50]:
def only_length_blind(split_file_name, length_model_name):
    from collections import Counter
    _, _,train_pairs,val_pairs,t_pairs=pickle.load(open(split_file_name,'rb'))
    t_pairs=blind_pairs
    length_model = pickle.load(open(length_model_name,'rb'))
    prefixes=[t.find('D') for t in zip(*train_pairs)[1]]
    cprefixes=Counter(prefixes)
    cpmf=Pmf(cprefixes)
    cpmf.normalize()
    cpmf_dist_keys, cpmf_dist_values=zip(*sorted(cpmf.items()))    
    #print cpmf_dist_keys, cpmf_dist_values
    preds=[]
    test_pairs=zip(*t_pairs)[0]
    for t in test_pairs:
        tlen, tstd=length_model.predict(np.array([len(t)]).reshape(-1,1), return_std=True)
        tlen_sample = int(np.random.normal(loc=tlen, scale=tstd))
        if tlen_sample > len(t):
            tlen_sample = len(t)
        len_prefix=np.random.choice(len(cpmf_dist_values), p=cpmf_dist_values)
        #print "Length", len_prefix
        pb, sb = t.split('}')
        p1blend=pb[:len_prefix]
        if len(p1blend) < tlen_sample:
            s1blend=sb[-(tlen_sample-len_prefix):]
        else:
            s1blend=''
        preds.append(p1blend+s1blend)
    gold_preds=[get_from_copy_edit(val_pair[0], val_pair[1]) for val_pair in t_pairs]
    edit_scores=[Levenshtein.distance(unicode(p),unicode(g)) for p, g in zip(preds, gold_preds)]

    return np.mean(edit_scores), preds, gold_preds, edit_scores, None

In [51]:
source_lang='components'
target_lang='blends-knight'
input_lang, output_lang, pairs = prepare_data(source_lang, target_lang, split_by_char_tokenizer, join_by_char_tokenizer, split_by_char_tokenizer, join_by_char_tokenizer, False)                                                                                                                                                                              
print output_lang.word2index                                                            

Reading lines...
Filtered to 397 pairs
Indexing words...
Indexed 30 words in input language, 5 words in output
Trimming languages
keep_words 27 / 27 = 1.0000
keep_words 2 / 2 = 1.0000
Read 397 sentence pairs
Trimmed from 397 pairs to 397, 1.0000 of total
Testing we can get a random batch.
Testing models
{u'C': 3, u'D': 4}


In [52]:
def get_edit_distance(embedding_dim, hidden_dim, vocab_size, tagset_size, batch_size, model_filename, split_file_name):
    m=load_model(embedding_dim=embedding_dim,hidden_dim=hidden_dim,vocab_size=vocab_size,tagset_size=tagset_size, batch_size=batch_size, model_filename=model_filename)
    _, _, _,val_pairs,t_pairs=pickle.load(open(split_file_name,'rb'))
    test_pairs=list(zip(*t_pairs)[0])
    preds=get_predict(m,test_pairs)
    gold_preds=[get_from_copy_edit(val_pair[0], val_pair[1]) for val_pair in t_pairs]
    edit_scores=[Levenshtein.distance(p,g) for p, g in zip(preds, gold_preds)]
    return np.mean(edit_scores), preds, gold_preds, edit_scores, m

# Only length

In [53]:
escores=[]
for fno in np.arange(0, 100):
    e,preds,gold_preds,edit_scores, _=only_length('./seq2seq_output/test_knight//split_lstm_{}'.format(fno), './seq2seq_output/test_knight_length/length_{}'.format(fno))
    escores.append(e)
print np.mean(escores)

2.4444999999999997


# Only LSTM + Greedy

In [54]:
escores=[]
for fno in np.arange(0, 100):
    e,p,gp,_, m=get_edit_distance(50,50,input_lang.n_words, output_lang.n_words, 20, './seq2seq_output/test_knight/lstm_{}_best'.format(fno),'./seq2seq_output/test_knight/split_lstm_{}'.format(fno))
    escores.append(e)
print np.mean(escores)

1.7594999999999998


In [55]:
import networkx as nx
import numpy as np
from itertools import product

In [56]:
def k_shortest_paths(G, source, target, k, weight=None):
    from itertools import islice
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))

def get_k_best(M, k=5):
    # We negate all things to find the best (longest). This only works because all entries in M are negative.
    #So making them positive is fine.
    G=nx.DiGraph()
    for r in np.arange(0, M.shape[0]-1):
        tuples=list(product([(r, c) for c in np.arange(0, M.shape[1])], [(r+1, c) for c in np.arange(0, M.shape[1])]))
        for t in tuples:
            G.add_edge(t[0], t[1], weight=-M[t[1][0], t[1][1]])

    for c in np.arange(0, M.shape[1]):
        G.add_edge(-1, (0,c), weight=-M[0][c])
        G.add_edge((M.shape[0]-1, c), M.shape[0], weight=0)
    
    tags=k_shortest_paths(G, -1, M.shape[0], k=k, weight='weight')
    return tags

def print_tags(M , tags):
    for tag in tags:
        print tag[1:-1], np.sum(M[cell[0],cell[1]] for cell in tag[1:-1])

In [57]:
def get_k_best_blends(M, s, k=5):
    N=M[:len(s)+1, :]
    hyps=get_k_best(N,k=k)
    candidates=[]
    for hyp in hyps:
        candidates.append((get_from_copy_edit(s, [output_lang.index2word[t] for t in zip(*hyp[1:-1])[1]]), np.sum(M[cell[0],cell[1]] for cell in hyp[1:-1])))
    return candidates

In [58]:
def get_edit_distance_lm(embedding_dim, hidden_dim, vocab_size, tagset_size, batch_size, model_filename, split_file_name, length_model_name, use_lm=False, use_pm=False, use_length=False):
    m=load_model(embedding_dim=embedding_dim,hidden_dim=hidden_dim,vocab_size=vocab_size,tagset_size=tagset_size, batch_size=batch_size, model_filename=model_filename)
    _, _, _,val_pairs,t_pairs=pickle.load(open(split_file_name,'rb'))
    length_model = pickle.load(open(length_model_name,'rb'))
    test_pairs=list(zip(*t_pairs)[0])
    preds, _=get_predict_lm(m,test_pairs, lmodel, length_model, use_lm=use_lm, use_pm=use_pm, use_length=use_length)
    gold_preds=[get_from_copy_edit(val_pair[0], val_pair[1]) for val_pair in t_pairs]
    edit_scores=[Levenshtein.distance(p,g) for p, g in zip(preds, gold_preds)]
    return np.mean(edit_scores), preds, gold_preds, edit_scores, m

## Only LSTM+Kbest

In [59]:
escores=[]
fold_dfs=[]
for fno in np.arange(0, 100):
    e,preds,gold_preds,edit_scores, m=get_edit_distance_lm(50,50,input_lang.n_words, output_lang.n_words, 20, './seq2seq_output/test_knight//lstm_{}_best'.format(fno),'./seq2seq_output/test_knight//split_lstm_{}'.format(fno), './seq2seq_output/test_knight_length/length_{}'.format(fno))
    escores.append(e)
    fdf=pd.DataFrame().from_records(zip(gold_preds, preds, edit_scores),columns=['g','p','s'])
    fold_dfs.append(fdf)
print np.mean(escores)

/home/vvkulkarni/postdoc/lib/python2.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


1.7594999999999998


## LSTM + LM 

In [60]:
escores=[]
fold_dfs_lm=[]
for fno in np.arange(0, 100):
    e,preds,gold_preds,edit_scores, m=get_edit_distance_lm(50,50,input_lang.n_words, output_lang.n_words, 20, './seq2seq_output/test_knight//lstm_{}_best'.format(fno),'./seq2seq_output/test_knight//split_lstm_{}'.format(fno), './seq2seq_output/test_knight_length/length_{}'.format(fno), use_lm=True)
    escores.append(e)
    fdf=pd.DataFrame().from_records(zip(gold_preds, preds, edit_scores),columns=['g','p','s'])
    fold_dfs_lm.append(fdf)
print np.mean(escores)

/home/vvkulkarni/postdoc/lib/python2.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


1.5257500000000002


## LSTM + LM + PM

In [61]:
escores=[]
fold_dfs_lm_pm=[]
for fno in np.arange(0, 100):
    e,preds,gold_preds,edit_scores, m=get_edit_distance_lm(50,50,input_lang.n_words, output_lang.n_words, 20, './seq2seq_output/test_knight//lstm_{}_best'.format(fno),'./seq2seq_output/test_knight//split_lstm_{}'.format(fno), './seq2seq_output/test_knight_length/length_{}'.format(fno), use_lm=True, use_pm=True)
    escores.append(e)
    fdf=pd.DataFrame().from_records(zip(gold_preds, preds, edit_scores),columns=['g','p','s'])
    fold_dfs_lm_pm.append(fdf)
print np.mean(escores)

/home/vvkulkarni/postdoc/lib/python2.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


1.4905000000000002


## LSTM + LM + PM + LEN 

In [62]:
escores=[]
fold_dfs_lm_pm_len=[]
for fno in np.arange(0, 100):
    e,preds,gold_preds,edit_scores, m=get_edit_distance_lm(50,50,input_lang.n_words, output_lang.n_words, 20, './seq2seq_output/test_knight//lstm_{}_best'.format(fno),'./seq2seq_output/test_knight//split_lstm_{}'.format(fno), './seq2seq_output/test_knight_length/length_{}'.format(fno), use_lm=True, use_pm=True, use_length=True)
    escores.append(e)
    fdf=pd.DataFrame().from_records(zip(gold_preds, preds, edit_scores),columns=['g','p','s'])
    fold_dfs_lm_pm_len.append(fdf)
print np.mean(escores)

/home/vvkulkarni/postdoc/lib/python2.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


1.4050000000000002


In [63]:
def get_scores_fold(fno):
    m=load_model(50,50,input_lang.n_words, output_lang.n_words, 20, './seq2seq_output/test_knight//lstm_{}_best'.format(fno))
    _, _, _,val_pairs,t_pairs=pickle.load(open('./seq2seq_output/test_knight//split_lstm_{}'.format(fno),'rb'))
    length_model = pickle.load(open('./seq2seq_output/test_knight_length/length_{}'.format(fno),'rb'))
    test_pairs=list(zip(*t_pairs)[0])
    preds, odf=get_predict_lm(m,test_pairs, lmodel, length_model=length_model)
    gold_preds=[get_from_copy_edit(val_pair[0], val_pair[1]) for val_pair in t_pairs]
    edit_scores=[Levenshtein.distance(p,g) for p, g in zip(preds, gold_preds)]
    fdf=pd.DataFrame().from_records(zip(gold_preds, preds, edit_scores),columns=['g','p','s'])
    return odf, fdf

In [64]:
def get_score_string(s, M, output_lang):
    scores=[]
    for i, a in enumerate(s):
        scores.append(M[i][output_lang.word2index[a]])
    scores.append(M[len(s)][EOS_token])
    return np.sum(scores)

def get_edit_distance_repr(components, blend):
    import difflib
    add=False
    output=[]
    for c in difflib.ndiff(components, blend):
        if c[0]=='-':
            output.append('D')
        elif c[0]=='+':
            add = True
            output.append(c[1:].strip())
        else:
            output.append('C')
    return ''.join(output), add

In [65]:
def get_predict_lm_ensemble(ensemble, examples, lmodel, length_model, alpha=0.5):  
    import itertools
    outputs=[]
    preds=[]
    for example in examples:
        ensemble_scores=[]
        for m in ensemble:
            m.hidden=m.init_hidden(batch_size=m.batch_size)
            t = encode_word(list(itertools.repeat(example, m.batch_size)))
            scores = m(t)
            scores=F.log_softmax(scores[0,:]).cpu().data.numpy()
            ensemble_scores.append(scores)
        scores = np.mean(ensemble_scores, axis=0) # Average all scores
        candidates, lstm_scores=zip(*get_k_best_blends(scores, example, k=20))
        pm_scores=[]
        for c in candidates:
            if c in phoneme_score_dict:
                pm_scores.append(phoneme_score_dict[c])
            else:
                pm_scores.append(0.0)
        length_scores=[score_candidate_length(c, example, length_model) for c in candidates]
        lm_scores=[lmodel.score(' '.join(list(c)))/(float(len(c))**alpha) for c in candidates]
        odf=pd.DataFrame().from_records(zip(candidates, lstm_scores,lm_scores, pm_scores, length_scores), columns=['candidate','lstm','lm', 'pm','lengthscore'])
        odf["total"]=odf["lstm"]+odf["lm"]+odf["pm"]+odf["lengthscore"]
        odf=odf.sort_values(by=['total'], ascending=False)
        odf=odf.reset_index()
        preds.append(odf.ix[0].candidate)
    return preds

In [66]:
def get_edit_distance_lm_ensemble(embedding_dim, hidden_dim, vocab_size, tagset_size, batch_size, ensemble_size, model_filename, split_file_name, length_model):
    ensemble=[]
    _, _, _,val_pairs,t_pairs=pickle.load(open(split_file_name,'rb'))
    test_pairs=list(zip(*t_pairs)[0])
    print test_pairs[:10]
    for e in np.arange(0, ensemble_size):
        model_filename = model_filename.format(e)
        m=load_model(embedding_dim=embedding_dim,hidden_dim=hidden_dim,vocab_size=vocab_size,tagset_size=tagset_size, batch_size=batch_size, model_filename=model_filename)
        ensemble.append(m)
    preds=get_predict_lm_ensemble(ensemble, test_pairs, lmodel, length_model)
    gold_preds=[get_from_copy_edit(val_pair[0], val_pair[1]) for val_pair in t_pairs]
    edit_scores=[Levenshtein.distance(p,g) for p, g in zip(preds, gold_preds)]
    return np.mean(edit_scores), preds, gold_preds, edit_scores, ensemble

In [67]:
embedding_dim=50
hidden_dim=50
vocab_size=input_lang.n_words
tagset_size=output_lang.n_words
batch_size=20
escores=[]
for fno in np.arange(0,10):
    print fno
    model_filename="./seq2seq_output/test_knight_ensemble///lstm_{}_{}_best".format(fno,'{}')
    split_file_name = "./seq2seq_output/test_knight_ensemble///split_lstm_{}_{}".format(fno, 0)
    length_model = pickle.load(open('./seq2seq_output/test_knight_length/length_{}'.format(fno),'rb'))
    e,_,_,_,_=get_edit_distance_lm_ensemble(embedding_dim, hidden_dim, vocab_size, tagset_size, batch_size, ensemble_size=50, model_filename=model_filename, split_file_name=split_file_name, length_model=length_model)
    escores.append(e)
print np.mean(escores), np.median(escores)

0
[u'mod}rocker', u'gerry}salamander', u'warui}luigi', u'cooperation}competition', u'goat}sheep', u'black}mexican', u'arise}ascent', u'malaya}asia', u'animation}electronics', u'fork}spoon']


/home/vvkulkarni/postdoc/lib/python2.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


1
[u'nacho}taco', u'tagalog}english', u'satisfactory}suffice', u'fruit}utopia', u'rubin}economics', u'satisfies}fries', u'frugal}google', u'video}blog', u'pin}interest', u'parking}arcade']
2
[u'odd}auditorium', u'fraud}audience', u'digital}repeater', u'whale}dolphin', u'wallaby}kangaroo', u'flavor}favorite', u'swahili}english', u'human}manure', u'motor}pedal', u'keyboard}guitar']
3
[u'chill}relax', u'web}episode', u'guy}eyeliner', u'dramatic}drastic', u'veritas}horizon', u'sex}expert', u'ebony}phonics', u'kentucky}indiana', u'smart}mark', u'sex}exile']
4
[u'rubin}economics', u'fork}spoon', u'michigan}indiana', u'frown}calculation', u'florida}alabama', u'stoned}drunk', u'sex}exile', u'drama}comedy', u'spectral}alert', u'pre}sequel']
5
[u'jeans}leggings', u'korean}english', u'frugal}google', u'intertwine}intermingle', u'banjo}guitar', u'capsule}tablet', u'stay}vacation', u'bayonet}network', u'fergie}delicious', u'tagalog}english']
6
[u'motor}hotel', u'beer}peerage', u'banana}toffee', u'b